# TaxiSimulation examples

- typical import

In [1]:
using TaxiSimulation, RoutingNetworks, JLD

## Creating TaxiProblems

The `TaxiProblem` class represents problems we are trying to solve. Its most important features are:
- a `Network` object from the `RoutingNetworks` package, together with a `RoutingPaths` object from the same package. These types represent the static routing graph and directions that will be used by taxis. Another `RoutingPaths` object describes the costs. The hypothesis is that taxis will use shortest paths in time (and not in cost)
- a set of `Customers` (all customers to appear)
- a set of `Taxis` (all taxis on the map)

###  Synthetic problems

- We create here a typical small-sized routing problem on a synthetic city

In [ ]:
# the network
n = urbanNetwork(8, distance=800.)

# the travel-times and paths. Here with road-types maximal speeds 
# (see RoutingNetworks for more details)
routing = roadTypeRouting(n)

#usually,the costs are proportional to the times (but not necessarily)
# here the cost is $5 per hour of driving
costs = RoutingPaths(n, routing.times*5./3600.)

# We create the taxi problem
# - customerTime = nb of seconds to pickup or dropoff a customer
# - waitingCost = nb of $ per second of a taxi waiting. (here $1/hour)
pb = TaxiProblem(n,routing,costs,customerTime= 10., waitingCost = 1./3600.)

# The problem is still incomplete, we need to add customer and taxis
# we first add random customers, see `taxiproblem/randomproblem.jl` for more info
# 1h of customer pickups, 0.35 customers per node per hour, fare = $80/hour, 
# customers can wait up to 5min after pickup time, and call 30min before
addRandomCustomers!(pb, 3600., 0.35, hourFare=80., custWait=5.0*60, custCall= 30.0*60)

# 20 uniformly distributed identical random taxis, all available at beginning of simulation
addRandomTaxis!(pb, 20)

# save this problem locally for latter use
save("data/smallurb.jld", "pb", pb)
pb

- We also create a harder medium-sized problem 

In [15]:
n = urbanNetwork(16, distance=300.)
r = roadTypeRouting(n)
costs = RoutingPaths(n,r.times*5./3600.)
pb = TaxiProblem(n,r,costs,customerTime= 10., waitingCost = 1./3600.)
addRandomCustomers!(pb, 3600., 0.35, hourFare=80., custWait=5.0*60, custCall= 30.0*60)
addRandomTaxis!(pb, 55)
pb = updateTcall(pb,60*20.0)
pb = updateTmax(pb, 60*5.0)
save("data/mediumurb.jld", "pb", pb)

### Real-world problems 

We present here the creation of a problem using Manhattan's taxi data

#### Constructing real-world networks
The first step is to create the desired routing network. This can be done with the package RoutingNetworks, from a polygon. In our case we will use the predefined MANHATTAN polygon.

In [ ]:
MANHATTAN_POLY = Tuple{Float32,Float32}[(-74.01369f0,40.69977f0), (-74.00597f0,40.702637f0), (-73.99944f0,40.70641f0), (-73.991714f0,40.708492f0), (-73.9761f0,40.71044f0), (-73.96923f0,40.72931f0), (-73.973526f0,40.736073f0), (-73.9615f0,40.75402f0), (-73.941765f0,40.774693f0), (-73.94348f0,40.78223f0), (-73.938156f0,40.78535f0), (-73.93593f0,40.79029f0), (-73.928894f0,40.79432f0), (-73.92872f0,40.803024f0), (-73.93318f0,40.80744f0), (-73.9349f0,40.833942f0), (-73.92134f0,40.85745f0), (-73.91893f0,40.858356f0), (-73.913956f0,40.863678f0), (-73.909706f0,40.872345f0), (-73.91829f0,40.875168f0), (-73.92648f0,40.879192f0), (-73.93344f0,40.87244f0), (-73.933525f0,40.86793f0), (-73.943436f0,40.853584f0), (-73.947945f0,40.85164f0), (-73.94713f0,40.84414f0), (-73.9552f0,40.828682f0), (-73.96091f0,40.8205f0), (-73.97734f0,40.79864f0), (-73.98957f0,40.78077f0), (-73.996994f0,40.770725f0), (-74.00352f0,40.761368f0), (-74.01064f0,40.75103f0), (-74.01532f0,40.719486f0), (-74.01764f0,40.719063f0), (-74.02047f0,40.704067f0)];

- get data (can take some time)

In [ ]:
manhattanNetwork = queryOsmPolygon(MANHATTAN_POLY)

- format the network to our need

In [ ]:
manhattanNetwork = roadTypeSubset(manhattanNetwork,1:6) # only keep main driving roads
manhattanNetwork = stronglyConnected(manhattanNetwork,1) # only keep the main connected component (hoping it's the one of node 1)
manhattanNetwork = intersections(manhattanNetwork) # simplify enormously the network, so that each node correspond to an intersection

- save the network to a julia JLD file for latter use

In [ ]:
saveTemplate(manhattanNetwork, "Manhattan")

- to load the network, just do:

In [ ]:
manhattanNetwork = loadTemplate("Manhattan")
# visualize(manhattanNetwork)

In [ ]:
visualize(manhattanNetwork)

#### Getting real taxi-data
For now, the package is able to parse and load CSVs of taxi trips from the NYC taxi and limousines commission. The CSVs can be found on their [website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). The data is parsed and files can then be saved by date to simplify the reading.

In [ ]:
# Download data from website (will take some time, size = 1.8gb)
const DATA_URL_04_2016 = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-04.csv"
download(DATA_URL_04_2016, "data/taxidata042016.csv")

In [ ]:
# Parse data
realcustomers = loadManhattanCustomers(manhattanNetwork, "data/taxidata042016.csv")

# Save to files (one per day)
saveByDate(realcustomers, "data/manhattan")

# clean a little bit
rm("data/taxidata042016.csv")

- Finally, let's create the taxi problem

In [ ]:
# let's get some nice routing data created using another project
times = load("data/timingsestimates042016-0911-weekdays.jld", "times");
# if the previous file doe not exist, create virtual times instead:
# times = roadTypeRouting(manhattanNetwork).times

routing = RoutingPaths(manhattanNetwork, times)
costs = RoutingPaths(manhattanNetwork, routing.times*5./3600.)
pb = TaxiProblem(manhattanNetwork, routing, costs, customerTime=30., waitingCost = 1./3600.)

# Data from Wednesday 6th of April 2016 between 9 and 10:30
custData = load("data/manhattan-2016-04-06.jld", "customers")
startTime = DateTime(2016, 4, 6, 9, 0, 0)
endTime = startTime + Dates.Minute(90)

# adding 100% of the real demand
addDataCustomers!(pb, custData, startTime, endTime, 1.0)

# adding 3000 taxis with a distribution matching the customers pickups (to avoid "border effect")
addDistributedTaxis!(pb, 3000)

# updating customer behavior: they book between 0 and 20min earlier and wait up to 5min for a taxi.
pb = updateTcall(pb, 60*20.0,random=true)
pb = updateTmax(pb, 60*5.0)
save("data/manhattan.jld", "pb", pb)

pb

# Offline Solving
Offline solving is about finding the optimal solution when all customers are known beforehand.

## Optimization-based solving

In [1]:
using TaxiSimulation, JLD

- We first solve the small problem to optimality, and visualize the result

In [2]:
pb = load("data/smallurb.jld", "pb")
offlineSol = mipSolve(pb, verbose=false)

Offline Solution, problem with 61 customers and 20 taxis
Profit : 849.24 dollars
7 customers rejected. 



mipSolve returns an `OfflineSolution` object. This object contains all the information to represent an offline solution. It basically contains the customers assigned to each taxi. A full `TaxiSolution` also contains all the taxi movements at any time. Full solutions can be used to visualize the result (though the conversion between the two types is implicit)
The commands of visualization are:
- `ARROWS` to move around
- `Z` and `X` to move in/out
- `A` and `S` to increase/decrease the drawing size
- `D` to show/hide the nodes of the network
- `Q` and `ESC` to quit
- `W` and `E` to slow down / accelerate time
- `R` to reverse time
- `SPACE` to pause time
- `click` to follow/unfollow a taxi

In [3]:
solution = TaxiSolution(offlineSol)
visualize(solution)

Solution metrics:
Revenues: $933.05, Costs: $83.81, Profit: $849.24 
88.52% of the demand is met
In total, 15.95h of driving
20.24% of waiting time, 21.19% of empty rides, 
Fairness: taxi profit = $42.46 +- $12.57


The problem is internally converted to a network-flow problem in order to be solved. This formulation creates a graph where each node is a customer, and each directed edge represents a pair of customers that can be picked-up in a row.
When all feasible edges are given, the solution is the optimal one. On the other hand, one can solve the problem with fewer edges for more tractability at the cost of a less optimal solution. 

Note that a `FlowProblem` must be fully computed (all feasible pair of customers). The graph indicates what edges we do consider.


In [5]:
pb = load("data/mediumurb.jld", "pb")
flowPb = FlowProblem(pb)

# compute heuristic score for each edge. It's a rough indicator of how good an edge is.
scores = scoreHeuristic(pb, flowPb)
# only select the 3 best edges for each nodes, and 10 for the taxi origin nodes
smallerPb = kLinks(flowPb, 3, scores, firstK=10)

# scores are useful to avoid recomputing everything for each new K.
# but there is a one-liner :
smallerPb = kLinks(pb, 2, firstK=10)


flowSol = mipFlow(smallerPb, verbose=false)
sol = OfflineSolution(pb, smallerPb, flowSol)
# visualize(sol)

In [6]:
sol

Offline Solution, problem with 272 customers and 55 taxis
Profit : 2896.18 dollars
44 customers rejected. 



Solution metrics:
Revenues: $3148.25, Costs: $252.07, Profit: $2896.18 
83.82% of the demand is met
In total, 49.27h of driving
10.42% of waiting time, 16.75% of empty rides, 
Fairness: taxi profit = $52.66 +- $20.13


When pick-up times are fixed, the problem becomes an LP and we can use a (fast) LP solver.

In [7]:
# random pickup times (can also provide other time window)
times = randPickupTimes(flowPb)

# all pick-up as late as possible
times = fixedPickupTimes(flowPb, 1.)

sol = OfflineSolution(pb, flowPb, lpFlow(flowPb, times, verbose=false))

Offline Solution, problem with 272 customers and 55 taxis
Profit : 2721.43 dollars
61 customers rejected. 



Solution metrics:
Revenues: $2968.85, Costs: $247.42, Profit: $2721.43 
77.57% of the demand is met
In total, 48.10h of driving
12.54% of waiting time, 18.60% of empty rides, 
Fairness: taxi profit = $49.48 +- $6.20


### Modifying a flow problem
It is possible to add links to a `FlowProblem` object:

In [10]:
# The easier way is to start with a fully precomputed `FlowProblem`:
flowPb = FlowProblem(pb)

# Then constructing a copy (share memory!) with the edges removed
newFlowPb = emptyFlow(flowPb)

# And add the edges of a computed solution:
flowSol = lpFlow(flowPb, fixedPickupTimes(flowPb), verbose=false)
addLinks!(newFlowPb, flowSol)

MIP flow network:
327 nodes - 211 edges


## Heuristics
- Greedy insertions

In [2]:
pb = load("data/mediumurb.jld", "pb")
orderedInsertions(pb) # order = tmin

Offline Solution, problem with 272 customers and 55 taxis
Profit : 2939.28 dollars
29 customers rejected. 



Solution metrics:
Revenues: $3204.51, Costs: $265.23, Profit: $2939.28 
89.34% of the demand is met
In total, 52.56h of driving
4.44% of waiting time, 19.21% of empty rides, 
Fairness: taxi profit = $53.44 +- $10.10


- local improvement of greedy insertions order

In [12]:
insertionsDescent(pb)

Try: 1, 2939.2825076070603 dollars
====Try: 660, 3038.55 dollars (0m8s, 4536.95 tests/min, 3.035% successful)   
Final profit: 3038.5479878714023 dollars


Offline Solution, problem with 272 customers and 55 taxis
Profit : 3038.55 dollars
27 customers rejected. 



Solution metrics:
Revenues: $3306.35, Costs: $267.80, Profit: $3038.55 
90.07% of the demand is met
In total, 53.20h of driving
3.27% of waiting time, 18.24% of empty rides, 
Fairness: taxi profit = $55.25 +- $9.29


- best of random insertions orders

In [4]:
sol = randomInsertions(pb)

Try: 1, 2407.980348015021 dollars
====Final: 2580.4285239004157 dollars              


Offline Solution, problem with 272 customers and 55 taxis
Profit : 2580.43 dollars
51 customers rejected. 



Solution metrics:
Revenues: $2835.82, Costs: $255.39, Profit: $2580.43 
81.25% of the demand is met
In total, 50.10h of driving
8.92% of waiting time, 22.63% of empty rides, 
Fairness: taxi profit = $46.92 +- $8.03


- local 2-OPT improvement of solution, with reinsertions at each step

In [5]:
localDescent(pb, sol, maxSearch=100, maxTime=20.)

====Try: 110630, 3126.90 dollars (0m19s, 343234.59 tests/min, 0.487% successful)   

Offline Solution, problem with 272 customers and 55 taxis
Profit : 3126.90 dollars
19 customers rejected. 



Solution metrics:
Revenues: $3395.34, Costs: $268.44, Profit: $3126.90 
93.01% of the demand is met
In total, 53.36h of driving
2.98% of waiting time, 16.99% of empty rides, 
Fairness: taxi profit = $56.85 +- $7.35


- For very large problems (eg. Manhattan), smart-search tunes the maxSearch parameter in an online way.

In [6]:
smartSearch(pb,sol)

$2898.46, 0m04s, 248/22623, 0.000% successful, search depth: 1(-1), update: 2.25s  No more improvements: stop                         


Offline Solution, problem with 272 customers and 55 taxis
Profit : 2898.46 dollars
28 customers rejected. 



Solution metrics:
Revenues: $3152.94, Costs: $254.48, Profit: $2898.46 
89.71% of the demand is met
In total, 49.87h of driving
9.33% of waiting time, 17.34% of empty rides, 
Fairness: taxi profit = $52.70 +- $8.97


## Online simulations

Online simulation relies on the `onlineSimulation` function, that takes a taxi problem and and feeds it to an online algorithm. Either updates the solution at a predefined time-steps or for each new customer.

A general class of online algorithm are offline-planning, that maintain an inner offline solution of future moves throughout the simulation

- The simplest one is `InsertOnly`, that just insert new customers into a taxi's timeline.

In [2]:
pb = load("data/mediumurb.jld", "pb")
algo = InsertOnly(earliest=false)
sol = onlineSimulation(pb, algo)

# a horizon can be added:
sol = onlineSimulation(pb, algo, horizon=60*5, verbose=false, period=30)

sim-time: 0m00s (0.00%) realTime:(0m00s)             


TaxiSolution, problem with 272 customers and 55 taxis
Profit : 2985.22 dollars
30 customers not served. 



- `NearestTaxi` assign the nearest free taxi to new customers, at the time they appear (pure online)

In [13]:
algo = NearestTaxi(freeTaxiOnly=false)
sol = onlineSimulation(pb, algo)

sim-time: 0m00s (0.00%) realTime:(0m00s)             


TaxiSolution, problem with 272 customers and 55 taxis
Profit : 2529.68 dollars
61 customers not served. 



Solution metrics:
Revenues: $2754.36, Costs: $224.68, Profit: $2529.68 
77.57% of the demand is met
1980.740km traveled, for 42.42h of driving
22.87% of waiting time, 15.85% of empty rides, 
Fairness: taxi profit = $45.99 +- $9.20


- `SearchBudget` is a heuristic-based algorithm that uses an offline solver to continuously improve the solution. The time_budget parameter controls the percentage of "simulation real-time" that is used to find a better solution. This method can use any offline solver.

In [5]:
algo = SearchBudget(time_budget = 0.01)
sol = onlineSimulation(pb, algo)

sim-time: 38m49s (98.68%) realTime:(0m23s)             


TaxiSolution, problem with 272 customers and 55 taxis
Profit : 2972.93 dollars
28 customers not served. 



Solution metrics:
Revenues: $3238.40, Costs: $265.47, Profit: $2972.93 
89.71% of the demand is met
2420.253km traveled, for 52.62h of driving
4.33% of waiting time, 18.74% of empty rides, 
Fairness: taxi profit = $54.05 +- $8.89


In [5]:
algo = SearchBudget(
update_solver     = (pb,init,custs,t)-> mipSolve(pb, init, verbose=false),
precompute_solver = (pb,custs)     -> mipSolve(pb, verbose=true),
    time_budget=1.,  update_freq=60.
)

sol = onlineSimulation(pb, algo, horizon=60*20)

pre-simulation computations...Optimize a model with 5055 rows, 5055 columns and 24075 nonzeros
Coefficient statistics:
  Matrix range    [4e-02, 1e+03]
  Objective range [3e-01, 2e+01]
  Bounds range    [1e+00, 2e+03]
  RHS range       [2e-01, 2e+03]
Found heuristic solution: objective -0
Presolve removed 3528 rows and 210 columns
Presolve time: 0.06s
Presolved: 1527 rows, 4845 columns, 12500 nonzeros
Variable types: 45 continuous, 4800 integer (4800 binary)

Root relaxation: objective 1.103924e+03, 426 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1103.92368    0   13   -0.00000 1103.92368      -     -    0s
H    0     0                    1103.6215484 1103.92368  0.03%     -    0s
H    0     0                    1103.7200099 1103.92368  0.02%     -    0s
     0     0 1103.88393    0   19 1103.72001 1103.88393  0.01%     -    0s
     0     0 1

TaxiSolution, problem with 272 customers and 55 taxis
Profit : 3315.49 dollars
57 customers not served. 



Solution metrics:
Revenues: $3603.94, Costs: $288.44, Profit: $3315.49 
95.59% of the demand is met
2731.608km traveled, for 58.36h of driving
-6.11% of waiting time, 18.57% of empty rides, 
Fairness: taxi profit = $60.28 +- $7.30
